In [1]:
import pandas as pd
import numpy as np
import time
from datetime import datetime

from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [79]:
input_ = []
s_time = time.time()
for i in range(6):
  input_.append(pd.read_excel('/gdrive/My Drive/input_'+str(i+1)+'.xlsx'))
print('Elapsed :', time.time() - s_time)

Elapsed : 45.88060474395752


In [80]:
# 전처리 1 : (연월종목 X 입력변수) 의 2차원 Dataframe 생성

# 6개 각각의 엑셀 파일을 2차원 dataframe으로 정리한 뒤 6개를 1개로 병합
new_input = []
for input_data in input_:
  for i in range(1,64): # 날짜 -> string
    input_data['Code'][i] = input_data['Code'][i].strftime('%Y%m')[2:]

  # 날짜 리스트
  date = input_data['Code'].tolist()[1:]

  num_input = 28 # 입력 변수 갯수
  num_stock = (input_data.shape[1] - 1)//num_input # 한 엑셀 파일당 종목 갯수
  num_date = len(date) # 기간 갯수

  # 종목별 데이터 쪼개기
  input_data_list = []
  for i in range(num_stock):
    input_data_list.append(input_1.iloc[:, num_input*i+1 : num_input*(i+1)+1])
  
  # 입력 변수 리스트
  input_list = input_data_list[0].loc[0,:].tolist()

  # 종목 당 (연월종목 X 입력변수) Dataframe 구성
  for i in range(num_stock):
    # 종목 코드 추출
    stock_code = input_data_list[i].columns[0]

    # 날짜 + 종목의 새 Index 구성
    new_date = [] 
    for t in date:
      new_date.append(t + stock_code)

    # Column 설정
    input_data_list[i].columns = input_list
    input_data_list[i].drop([0], inplace=True)

    # Index 설정
    input_data_list[i].index = new_date
  
  # 종목 Dataframe 결합
  new_input.append(pd.concat(input_data_list))
  
# 엑셀 별 Dataframe 결합
all_input = pd.concat(new_input)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [81]:
all_input

,수정주가,월간평균거래량,월간수익률,20일거래량회전율,60일변동성(표준편차),20일평균대차거래체결량비중,월간평균 공매도수량/거래량,외국인지분율,유동주식비율(보통주),유동자산,비유동자산,부채총계,자본금,기타포괄이익누계액,비지배주주지분,Cash Earnings,영업활동으로인한현금흐름,투자활동으로인한현금흐름,CAPEX,배당금,자본의증감,현금의증가,매출원가,매출총이익,판매비와관리비,인건비및복리후생비,영업이익,당기순이익
1504A000020,8330,218978,10.92,0.007854,0.020248,0,0.72,6.36,61.99,1.35015e+08,1.82172e+08,8.70693e+07,27931470,2.94674e+06,NaN,1.74731e+07,1.98331e+07,-979732,885504,2.2345e+06,-2.35441e+06,1.22662e+07,1.26622e+08,9.65794e+07,9.17664e+07,31392013,4.81297e+06,5.60865e+06
1505A000020,9370,332579,12.48,0.010915,0.029854,0,0.4,7,61.99,1.35015e+08,1.82172e+08,8.70693e+07,27931470,2.94674e+06,NaN,1.74731e+07,1.98331e+07,-979732,885504,2.2345e+06,-2.35441e+06,1.22662e+07,1.26622e+08,9.65794e+07,9.17664e+07,31392013,4.81297e+06,5.60865e+06
1506A000020,9070,295146,-3.2,0.008875,0.033701,0,0.88,8.21,61.99,1.35015e+08,1.82172e+08,8.70693e+07,27931470,2.94674e+06,NaN,1.74731e+07,1.98331e+07,-979732,885504,2.2345e+06,-2.35441e+06,1.22662e+07,1.26622e+08,9.65794e+07,9.17664e+07,31392013,4.81297e+06,5.60865e+06
1507A000020,8160,596820,-10.03,0.017638,0.040963,0,1.39,8.54,61.99,1.35015e+08,1.82172e+08,8.70693e+07,27931470,2.94674e+06,NaN,1.74731e+07,1.98331e+07,-979732,885504,2.2345e+06,-2.35441e+06,1.22662e+07,1.26622e+08,9.65794e+07,9.17664e+07,31392013,4.81297e+06,5.60865e+06
1508A000020,7100,130112,-12.99,0.004658,0.036095,0.02,2.45,8.56,61.99,1.35015e+08,1.82172e+08,8.70693e+07,27931470,2.94674e+06,NaN,1.74731e+07,1.98331e+07,-979732,885504,2.2345e+06,-2.35441e+06,1.22662e+07,1.26622e+08,9.65794e+07,9.17664e+07,31392013,4.81297e+06,5.60865e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2002A004780,3710,62124,-7.83,0.003906,0.020517,0.02,4.9,1.42,43.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003A004780,3190,110005,-14.02,0.007439,0.034819,0.02,1.23,1.61,43.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004A004780,3900,109578,22.26,0.00689,0.034882,0,NaN,2.14,43.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005A004780,4105,95213,5.26,0.005988,0.034824,0,NaN,2.31,43.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
# 전처리 2 : 결측 값 처리